# Drug Name Entity Classifier
## AHLT - MIRI 2018



## Initialization

Load needed modules and specify the working directory

In [1]:
# Load needed packages
import pandas as pd
import numpy as np
import scipy


import time # Execution time of some blocks
from nltk.tag import StanfordPOSTagger

# sklearn package
from sklearn.metrics import make_scorer
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics


# Import our defined functions
from NER_functions import *
from datasetBuilder import *

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# init time
first_init = time.time()

In [3]:
# Set the data directories
# train_dirs_whereto_parse = ['data/medium_train_DrugBank']
# test_dirs_whereto_parse = ['data/medium_test_DrugBank']
train_dirs_whereto_parse = ['data/Train/DrugBank','data/Train/MedLine']
test_dirs_whereto_parse = ['data/Test/Test for DrugNER task/DrugBank','data/Test/Test for DrugNER task/MedLine']

## Reading the train and test data from the XML files
Accessing to all the files of the directory and storing id's and text's in two arrays.
We have also added the token 'STOP' at the end of each sentence.

In [4]:
train_texts_entities = createSet(train_dirs_whereto_parse)
test_texts_entities = createSet(test_dirs_whereto_parse)

Before computing features, I want the input data to have a special format:

In [5]:
# we want each dataset with the following format: 
# for ['have','Ibuprofeno'] ---- [('hola','V','O'),('Ibuprofeno','NN','B')]

def buildSet(text_entities):
    dataset = []
    for text,drugs in text_entities:
        # tokenizing
        tokenized_sentence = nltk.word_tokenize(text)
        # BIO tagging
        tokens_tags = BIOTagger(text, drugs)
        # POS tagging
        tokens_pos = nltk.pos_tag(tokenized_sentence)

        text_triples = []
        for idx,token in enumerate(tokenized_sentence):
            text_triples.append((token,tokens_pos[idx][1],tokens_tags[idx][1]))
        dataset.append(text_triples)
    
    return dataset

# train_set and test_set are list of list of triples; each list of triples refers to a different sentence
train_set = buildSet(train_texts_entities) 
test_set = buildSet(test_texts_entities)

print('Number of training sentences: ',len(train_set))
print('Num of test sentences: ',len(test_set))

print(test_set)

drugs multiples:  []
[('Milk', 'O'), (',', 'O'), ('milk', 'O'), ('products', 'O'), (',', 'O'), ('and', 'O'), ('calcium-rich', 'O'), ('foods', 'O'), ('or', 'O'), ('drugs', 'O'), ('may', 'O'), ('impair', 'O'), ('the', 'O'), ('absorption', 'O'), ('of', 'O'), ('EMCYT', 'B-brand'), ('.', 'O')]
drugs multiples:  []
[('The', 'O'), ('concurrent', 'O'), ('administration', 'O'), ('of', 'O'), ('allopurinol', 'B-drug'), ('and', 'O'), ('ampicillin', 'B-drug'), ('increases', 'O'), ('substantially', 'O'), ('the', 'O'), ('incidence', 'O'), ('of', 'O'), ('rashes', 'O'), ('in', 'O'), ('patients', 'O'), ('receiving', 'O'), ('both', 'O'), ('drugs', 'O'), ('as', 'O'), ('compared', 'O'), ('to', 'O'), ('patients', 'O'), ('receiving', 'O'), ('ampicillin', 'B-drug'), ('alone', 'O'), ('.', 'O')]
drugs multiples:  []
[('It', 'O'), ('is', 'O'), ('not', 'O'), ('known', 'O'), ('whether', 'O'), ('this', 'O'), ('potentiation', 'O'), ('of', 'O'), ('ampicillin', 'B-drug'), ('rashes', 'O'), ('is', 'O'), ('due', 'O'), ('

drugs multiples:  []
[('Table', 'O'), ('1', 'O'), ('Changes', 'O'), ('in', 'O'), ('Desloratadine', 'B-drug'), ('and', 'O'), ('3-Hydroxydesloratadine', 'B-drug_n'), ('Pharmacokinetics', 'O'), ('in', 'O'), ('Healthy', 'O'), ('Male', 'O'), ('and', 'O'), ('Female', 'O'), ('Volunteers', 'O')]
drugs multiples:  []
[('Desloratadine', 'B-drug'), ('3-Hydroxydesloratadine', 'B-drug_n')]
drugs multiples:  []
[('Erythromycin', 'B-drug')]
drugs multiples:  []
[('Ketoconazole', 'B-drug')]
drugs multiples:  []
[('Azithromycin', 'B-drug')]
drugs multiples:  []
[('Fluoxetine', 'B-drug')]
drugs multiples:  []
[('Cimetidine', 'B-drug')]
drugs multiples:  []
[('Enoxaparin', 'B-drug'), ('dosed', 'O'), ('as', 'O'), ('a', 'O'), ('1.0', 'O'), ('mg/kg', 'O'), ('subcutaneous', 'O'), ('injection', 'O'), ('q12h', 'O'), ('for', 'O'), ('four', 'O'), ('doses', 'O'), ('did', 'O'), ('not', 'O'), ('alter', 'O'), ('the', 'O'), ('pharmacokinetics', 'O'), ('of', 'O'), ('eptifibatide', 'B-drug'), ('or', 'O'), ('the', 'O'),

drugs multiples:  [(['TNF', 'blocking', 'agents'], 'group')]
[('Population', 'O'), ('pharmacokinetic', 'O'), ('analyses', 'O'), ('revealed', 'O'), ('that', 'O'), ('MTX', 'B-drug'), (',', 'O'), ('NSAIDs', 'B-group'), (',', 'O'), ('corticosteroids', 'B-group'), (',', 'O'), ('and', 'O'), ('TNF', 'B-group'), ('blocking', 'I'), ('agents', 'I'), ('did', 'O'), ('not', 'O'), ('influence', 'O'), ('abatacept', 'O'), ('clearance', 'B-drug'), ('.', 'O')]
drugs multiples:  [(['TNF', 'blocking', 'agents'], 'group')]
[('The', 'O'), ('majority', 'O'), ('of', 'O'), ('patients', 'O'), ('in', 'O'), ('RA', 'O'), ('clinical', 'O'), ('studies', 'O'), ('received', 'O'), ('one', 'O'), ('or', 'O'), ('more', 'O'), ('of', 'O'), ('the', 'O'), ('following', 'O'), ('concomitant', 'O'), ('medications', 'O'), ('with', 'O'), ('ORENCIA', 'B-brand'), (':', 'O'), ('MTX', 'B-drug'), (',', 'O'), ('NSAIDs', 'B-group'), (',', 'O'), ('corticosteroids', 'B-group'), (',', 'O'), ('TNF', 'B-group'), ('blocking', 'I'), ('agents', 

drugs multiples:  []
[('Prothrombin', 'O'), ('times', 'O'), ('were', 'O'), ('not', 'O'), ('affected', 'O'), ('by', 'O'), ('tiagabine', 'B-drug'), ('.', 'O')]
drugs multiples:  []
[('Digoxin', 'B-drug'), (':', 'O'), ('Concomitant', 'O'), ('administration', 'O'), ('of', 'O'), ('tiagabine', 'B-drug'), ('did', 'O'), ('not', 'O'), ('affect', 'O'), ('the', 'O'), ('steady-state', 'O'), ('pharmacokinetics', 'O'), ('of', 'O'), ('digoxin', 'B-drug'), ('or', 'O'), ('the', 'O'), ('mean', 'O'), ('daily', 'O'), ('trough', 'O'), ('serum', 'O'), ('level', 'O'), ('of', 'O'), ('digoxin', 'B-drug'), ('.', 'O')]
drugs multiples:  []
[('Ethanol', 'B-drug'), ('or', 'O'), ('Triazolam', 'B-drug'), (':', 'O'), ('No', 'O'), ('significant', 'O'), ('differences', 'O'), ('were', 'O'), ('observed', 'O'), ('in', 'O'), ('the', 'O'), ('pharmacokinetics', 'O'), ('of', 'O'), ('triazolam', 'B-drug'), ('(', 'O'), ('0.125', 'O'), ('mg', 'O'), (')', 'O'), ('and', 'O'), ('tiagabine', 'B-drug'), ('(', 'O'), ('10', 'O'), ('mg'

drugs multiples:  []
[('Drug', 'O'), ('/', 'O'), ('Laboratory', 'O'), ('Test', 'O'), ('Interactions', 'O'), ('As', 'O'), ('a', 'O'), ('result', 'O'), ('of', 'O'), ('administration', 'O'), ('of', 'O'), ('Keflex', 'B-brand'), (',', 'O'), ('a', 'O'), ('false-positive', 'O'), ('reaction', 'O'), ('for', 'O'), ('glucose', 'O'), ('in', 'O'), ('the', 'O'), ('urine', 'O'), ('may', 'O'), ('occur', 'O'), ('.', 'O')]
drugs multiples:  [(['levamisole', 'hydrochloride'], 'drug')]
[('Caution', 'O'), ('should', 'O'), ('be', 'O'), ('exercised', 'O'), ('when', 'O'), ('administering', 'O'), ('ETOPOPHOS', 'B-brand'), ('with', 'O'), ('drugs', 'O'), ('that', 'O'), ('are', 'O'), ('known', 'O'), ('to', 'O'), ('inhibit', 'O'), ('phosphatase', 'O'), ('activities', 'O'), ('(', 'O'), ('e.g.', 'O'), (',', 'O'), ('levamisole', 'B-drug'), ('hydrochloride', 'I'), (')', 'O'), ('.', 'O')]
drugs multiples:  [(['cyclosporin', 'A'], 'drug')]
[('High-dose', 'O'), ('cyclosporin', 'B-drug'), ('A', 'I'), ('resulting', 'O'), (

...

drugs multiples:  []
[('In', 'O'), ('vitro', 'O'), ('activity', 'O'), ('of', 'O'), ('KRM-1648', 'B-drug_n'), (',', 'O'), ('either', 'O'), ('singly', 'O'), ('or', 'O'), ('in', 'O'), ('combination', 'O'), ('with', 'O'), ('ofloxacin', 'B-drug'), (',', 'O'), ('against', 'O'), ('Mycobacterium', 'O'), ('ulcerans', 'O'), ('.', 'O')]
drugs multiples:  []
[('The', 'O'), ('antimicrobial', 'O'), ('effect', 'O'), ('of', 'O'), ('a', 'O'), ('benzoxazinorifamycin', 'B-drug_n'), (',', 'O'), ('KRM-1648', 'B-drug_n'), (',', 'O'), ('either', 'O'), ('alone', 'O'), ('or', 'O'), ('in', 'O'), ('combination', 'O'), ('with', 'O'), ('ofloxacin', 'B-drug'), (',', 'O'), ('was', 'O'), ('evaluated', 'O'), ('in', 'O'), ('vitro', 'O'), ('against', 'O'), ('two', 'O'), ('type', 'O'), ('strains', 'O'), ('and', 'O'), ('six', 'O'), ('clinical', 'O'), ('isolates', 'O'), ('of', 'O'), ('Mycobacterium', 'O'), ('ulcerans', 'O'), ('.', 'O')]
drugs multiples:  []
[('The', 'O'), ('minimal', 'O'), ('inhibitory', 'O'), ('concentrat

drugs multiples:  [(['sodium', 'bicarbonate'], 'drug')]
[('He', 'O'), ('was', 'O'), ('treated', 'O'), ('aggressively', 'O'), ('with', 'O'), ('intravenous', 'O'), ('hydration', 'O'), (',', 'O'), ('sodium', 'B-drug'), ('bicarbonate', 'I'), (',', 'O'), ('and', 'O'), ('hemodialysis', 'O'), ('.', 'O')]
drugs multiples:  []
[('DISCUSSION', 'O'), (':', 'O'), ('Clarithromycin', 'B-drug'), ('is', 'O'), ('a', 'O'), ('potent', 'O'), ('inhibitor', 'O'), ('of', 'O'), ('CYP3A4', 'O'), (',', 'O'), ('the', 'O'), ('major', 'O'), ('enzyme', 'O'), ('responsible', 'O'), ('for', 'O'), ('simvastatin', 'B-drug'), ('metabolism', 'O'), ('.', 'O')]
drugs multiples:  [(['macrolide', 'antibiotics'], 'group'), (['hydroxymethylglutaryl', 'coenzyme', 'A', '(HMG-CoA)', 'reductase', 'inhibitors'], 'group')]
[('The', 'O'), ('concomitant', 'O'), ('administration', 'O'), ('of', 'O'), ('macrolide', 'B-group'), ('antibiotics', 'I'), ('and', 'O'), ('other', 'O'), ('hydroxymethylglutaryl', 'B-group'), ('coenzyme', 'I'), ('A'

drugs multiples:  []
[('Alcohol', 'B-drug'), ('(', 'O'), ('this', 'O'), ('combination', 'O'), ('may', 'O'), ('make', 'O'), ('you', 'O'), ('very', 'O'), ('sick', 'O'), (')', 'O'), ('and', 'O'), ('primaquine', 'B-drug')]
drugs multiples:  [(['methyl', 'aminolevulinate'], 'drug')]
[('There', 'O'), ('have', 'O'), ('been', 'O'), ('no', 'O'), ('studies', 'O'), ('of', 'O'), ('the', 'O'), ('interaction', 'O'), ('of', 'O'), ('methyl', 'B-drug'), ('aminolevulinate', 'I'), ('cream', 'O'), ('with', 'O'), ('any', 'O'), ('other', 'O'), ('drugs', 'O'), (',', 'O'), ('including', 'O'), ('local', 'O'), ('anesthetics', 'B-group'), ('.', 'O')]
drugs multiples:  [(['methyl', 'aminolevulinate'], 'drug')]
[('It', 'O'), ('is', 'O'), ('possible', 'O'), ('that', 'O'), ('concomitant', 'O'), ('use', 'O'), ('of', 'O'), ('other', 'O'), ('known', 'O'), ('photosensitizing', 'O'), ('agents', 'O'), ('might', 'O'), ('increase', 'O'), ('the', 'O'), ('photosensitivity', 'O'), ('reaction', 'O'), ('of', 'O'), ('actinic', 'O

drugs multiples:  []
[('Polymorphic', 'O'), ('enzymes', 'O'), ('are', 'O'), ('involved', 'O'), ('in', 'O'), ('phase-I-metabolism', 'O'), ('of', 'O'), ('THC', 'B-drug_n'), ('.', 'O')]
drugs multiples:  []
[('Pharmacological', 'O'), ('effects', 'O'), ('of', 'O'), ('other', 'O'), ('cannabinoids', 'B-drug_n'), ('or', 'O'), ('medication', 'O'), ('on', 'O'), ('the', 'O'), ('pharmacokinetics', 'O'), ('of', 'O'), ('THC', 'B-drug_n'), ('have', 'O'), ('not', 'O'), ('yet', 'O'), ('been', 'O'), ('studied', 'O'), ('in', 'O'), ('detail', 'O'), ('.', 'O')]
drugs multiples:  []
[('knowledge', 'O'), ('on', 'O'), ('the', 'O'), ('stability', 'O'), ('of', 'O'), ('cannabinoids', 'B-drug_n'), ('in', 'O'), ('forensic', 'O'), ('specimens', 'O'), ('is', 'O'), ('still', 'O'), ('poor', 'O'), ('.', 'O')]
drugs multiples:  []
[('Whether', 'O'), ('determination', 'O'), ('of', 'O'), ('cannabinoid', 'B-drug_n'), ('conjugates', 'O'), ('may', 'O'), ('be', 'O'), ('useful', 'O'), ('is', 'O'), ('a', 'O'), ('matter', 'O'),

drugs multiples:  [(['antiplatelet', 'agent'], 'group')]
[('In', 'O'), ('clinical', 'O'), ('trials', 'O'), ('the', 'O'), ('antiplatelet', 'B-group'), ('agent', 'I'), ('reduced', 'O'), ('significantly', 'O'), ('vascular', 'O'), ('mortality', 'O'), ('and', 'O'), ('death', 'O'), ('from', 'O'), ('any', 'O'), ('cause', 'O'), ('when', 'O'), ('compared', 'O'), ('to', 'O'), ('clopidogrel', 'B-drug'), ('in', 'O'), ('patients', 'O'), ('with', 'O'), ('acute', 'O'), ('coronary', 'O'), ('syndrome', 'O'), ('.', 'O')]
drugs multiples:  []
[('Unrecognized', 'O'), ('fatalities', 'O'), ('related', 'O'), ('to', 'O'), ('colchicine', 'B-drug'), ('in', 'O'), ('hospitalized', 'O'), ('patients', 'O'), ('.', 'O')]
drugs multiples:  []
[('Colchicine', 'B-drug'), ('is', 'O'), ('commonly', 'O'), ('used', 'O'), ('for', 'O'), ('the', 'O'), ('treatment', 'O'), ('of', 'O'), ('gout', 'O'), ('and', 'O'), ('occasionally', 'O'), ('for', 'O'), ('other', 'O'), ('inflammatory', 'O'), ('diseases', 'O'), ('.', 'O')]
drugs mul

## FEATURES 

In [6]:
with(open('data/DrugBank_names_DB.txt', 'r')) as f:
    drugbank_db = f.read().splitlines()
    
def word2features(sent, i, database):
    word = sent[i][0]
    postag = sent[i][1]
    
    # orthographic features
    features = {
    'all_uppercase_letters' : allCaps(word), 
    'initial_capital_letter': initCap(word), 
    'contains_capital_letter' : hasCap(word),
    'single_capital_letter' : singleCap(word),
    'punctuation' : punctuation(word),
    'initial_digit' : initDigit(word),
    'single_digit' : singleDigit(word),
    'letter_and_num' : alphaNum(word),
    'many_numbers' : manyNum(word),
    'contains_real_numbers' : realNum(word),
    'intermediate_dash' : inDash(word),
    'has_digit' : hasDigit(word),
    'is_Dash' : isDash(word),
    'is_roman_letter' : roman(word),
    'is_end_punctuation' : endPunctuation(word),
    'caps_mix' : capsMix(word),
        
    # Morphological information: prefixes/suffixes of lengths from 2 to 5 and word shapes of tokens. 
    'word[-5:]': word[-5:],
    'word[-4:]': word[-4:],    
    'word[-3:]': word[-3:],
    'word[-2:]': word[-2:],
    
    # Domain knowledge
    'contains_drug_sufix': containsSufix(word),
    'contains_drug_prefix': containsPrefix(word),

    # POS tag   
    'postag': postag,
    'postag[:2]': postag[:2],
        
    # Is in DrugBank dataset
    'isInDB':isTokenInDB(word,database)
    }
    
    
    
    # context features
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:unigrams' : num_ngrams(word1,1),
            '-1:bigrams' : num_ngrams(word1,2),
            '-1:trigrams' : num_ngrams(word1,3)
        })
        if i!=1:
            word2 = sent[i-2][0]
            postag2 = sent[i-2][1]
            features.update({
            '-2:postag': postag2,
            '-2:postag[:2]': postag2[:2],
            '-2:unigrams' : num_ngrams(word2,1)})
            if len(word2)>2:
                features.update({
                    '-2:bigrams' : num_ngrams(word2,2),
                    '-2:trigrams' : num_ngrams(word2,3)})
            elif len(word2)==2:
                features.update({
                    '-2:bigrams' : num_ngrams(word2,2)})
                
            
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:unigrams' : num_ngrams(word1,1),
            '+1:bigrams' : num_ngrams(word1,2),
            '+1:trigrams' : num_ngrams(word1,3)
        })
        
        if i<len(sent)-2:
            word2 = sent[i+2][0]
            postag2 = sent[i+2][1]
            features.update({
                '+2:postag': postag2,
                '+2:postag[:2]': postag2[:2],
                '+2:unigrams' : num_ngrams(word2,1)})
            if len(word2)>2:
                features.update({
                    '+2:bigrams' : num_ngrams(word2,2),
                    '+2:trigrams' : num_ngrams(word2,3)})
            elif len(word2)==2:
                features.update({
                    '+2:bigrams' : num_ngrams(word2,2)})
            
    else:
        features['EOS'] = True

    return features


def sent2features(sent,database):
    return [word2features(sent, i,drugbank_db) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [7]:
 # remember s is a triple defined as follows: (token, POS tag, BIO tag)
X_train = [sent2features(s,drugbank_db) for s in train_set]
y_train = [sent2labels(s) for s in train_set]

X_test = [sent2features(s,drugbank_db) for s in test_set]
y_test = [sent2labels(s) for s in test_set]


In [8]:
print(X_test)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
y_test

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-brand', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-brand',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-group',
  'I',
  'O'],
 ['B-drug', 'O', 'B-drug', 'O'],
 ['B-drug', 'O', 'B-drug', 'O'],
 ['B-drug', 'I', 'O', 'B-drug', 'O'],
 ['B-drug', 'O', 'B-drug', 'O', 'B-drug', 'O', 'B-drug', 'O'],
 ['B-drug', 'O'],
 ['B-drug', 'O', 'B-drug', 'O', 'B-drug', 'O', 'B-drug', 'O'],
 ['O', 'B-drug', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-drug',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-brand',
  'O'],
 ['O', 'O', 'B-drug', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-brand', 'O'],
 ['O',
  'O',
  'O',
  'B-drug',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-drug',
  'O',
  'O',
  

In [18]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
'''
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}
'''
# use the same metric for evaluation
labels = list(set(sum(y_train,[])))
labels.remove('O')
'''
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=30,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)
'''

"\nf1_scorer = make_scorer(metrics.flat_f1_score,\n                        average='weighted', labels=labels)\n\nrs = RandomizedSearchCV(crf, params_space,\n                        cv=3,\n                        verbose=1,\n                        n_jobs=-1,\n                        n_iter=30,\n                        scoring=f1_scorer)\nrs.fit(X_train, y_train)\n"

####  Predictions

In [14]:
#model = rs.best_estimator_
model = crf
model.fit(X_train,y_train)
pred_tags = model.predict(X_test)

####  Evaluation

####  Evaluation with scickit learn

In [19]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, pred_tags, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

          I      0.750     0.544     0.630       160
    B-brand      0.550     0.569     0.559        58
     B-drug      0.735     0.825     0.778       337
   B-drug_n      0.500     0.036     0.067       111
    B-group      0.683     0.638     0.660       149

avg / total      0.684     0.610     0.615       815



In [15]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(model.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(model.transition_features_).most_common()[-20:])

Top likely transitions:
B-group -> I       2.078505
O      -> O       1.739502
B-drug_n -> I       1.367041
O      -> B-drug  0.587624
O      -> B-group 0.498394
I      -> I       0.412162
B-drug -> O       0.331569
B-drug -> I       0.231301
I      -> O       0.178906
O      -> B-brand 0.078913
B-brand -> O       0.057281
O      -> B-drug_n -0.097167
B-drug_n -> O       -0.431840
B-brand -> I       -0.920666
B-group -> O       -1.342953
B-drug_n -> B-drug_n -1.961197
B-brand -> B-drug_n -2.045809
B-group -> B-drug_n -2.425597
I      -> B-drug_n -2.610002
B-drug_n -> B-brand -2.738930

Top unlikely transitions:
B-brand -> B-drug_n -2.045809
B-group -> B-drug_n -2.425597
I      -> B-drug_n -2.610002
B-drug_n -> B-brand -2.738930
B-drug_n -> B-group -2.895922
B-group -> B-brand -3.878620
B-drug -> B-drug_n -3.926334
B-brand -> B-group -4.024678
I      -> B-group -4.043130
I      -> B-brand -4.048606
I      -> B-drug  -4.100771
B-drug -> B-group -4.813463
B-drug_n -> B-drug  -5.184780
B-g

In [16]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(model.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(model.state_features_).most_common()[-30:])

Top positive:
4.664937 B-brand  word[-5:]:pirin
4.248495 B-group  word[-2:]:Is
4.136248 B-drug_n word[-5:]:gaine
4.058356 B-group  word[-2:]:Ds
3.864325 B-drug   word[-5:]:goxin
3.730101 B-drug_n word[-2:]:MC
3.707979 B-group  word[-5:]:ylate
3.693886 B-group  word[-5:]:piate
3.689163 B-group  word[-5:]:ators
3.605284 B-group  word[-5:]:zides
3.511948 I        -2:trigrams:spa
3.355780 B-group  word[-5:]:tacid
3.294647 B-brand  word[-2:]:EX
3.262713 O        BOS
3.243066 B-drug_n word[-5:]:atrol
3.219079 B-group  word[-4:]:tins
3.159221 B-drug   word[-5:]:ption
3.125906 B-group  word[-5:]:ogens
3.124570 B-drug_n word[-5:]:toxin
3.102012 B-drug_n word[-5:]:enate
3.065746 O        EOS
2.890748 B-drug   -1:trigrams:nsu
2.862509 B-drug   isInDB
2.861816 B-brand  word[-2:]:OL
2.849857 O        word[-5:]:rnate
2.822979 B-group  word[-5:]:marin
2.808355 B-brand  word[-2:]:AN
2.792856 B-brand  word[-2:]:AR
2.736668 O        -1:trigrams:ara
2.731855 O        +2:trigrams:Mac

Top negative:
-2.100

In [ ]:
print('Total execution time: ',(time.time() - first_init)/60, ' minutes')

In [ ]:

'''
## Log of results
date, precision, recall, F1, features, test
14-May, 46.2, 52.1, 48.99, Token length; Prefixes/Suffixes; POS tag; Binary features (+-2); Token position; DrugBank DB; Shape, yes
'''